## Time Series Analysis
1. Loading data
2. create X_short, a matrix with a single software per date
3. fit VAR(n) and predict m software
4. evaluate via F-score

Later the aggregation is performed but not further investigated.

In [3]:
import pandas as pd 
import numpy as np

In [4]:
X = pd.read_excel("../Data for algorithms.xlsx", sheet_name="X", index_col="ID").T
X = X.drop(["prop"], axis=1)
X

ID                       S0066                S0002                S0051  \
created    2017-05-31 00:00:00  2017-05-31 00:00:00  2017-05-31 00:00:00   
T1548.002                    0                    0                    0   
T1134                        0                    0                    0   
T1134.002                    0                    0                    0   
T1134.001                    0                    0                    0   
...                        ...                  ...                  ...   
T1102                        0                    0                    0   
T1102.002                    0                    0                    0   
T1102.001                    0                    0                    1   
T1102.003                    0                    0                    0   
T1047                        0                    0                    0   

ID                       S0083                S0080                S0079  \
created    2017-05-31 00:00:00  2017-05-31 00:00:00  2017-05-31 00:00:00   
T1548.002                    0                    0                    0   
T1134                        0                    0                    0   
T1134.002                    0                    0                    0   
T1134.001                    0                    0                    0   
...                        ...                  ...                  ...   
T1102                        0                    0                    0   
T1102.002                    0                    0                    0   
T1102.001                    0                    0                    0   
T1102.003                    0                    0                    0   
T1047                        0                    0                    0   

ID                       S0149                S0034                S0039  \
created    2017-05-31 00:00:00  2017-05-31 00:00:00  2017-05-31 00:00:00   
T1548.002                    0                    0                    0   
T1134                        0                    0                    0   
T1134.002                    0                    0                    0   
T1134.001                    0                    0                    0   
...                        ...                  ...                  ...   
T1102                        0                    0                    0   
T1102.002                    0                    0                    0   
T1102.001                    0                    0                    0   
T1102.003                    0                    0                    0   
T1047                        0                    0                    0   

ID                       S0056  ...                S0458                S0460  \
created    2017-05-31 00:00:00  ...  2020-05-27 00:00:00  2020-05-29 00:00:00   
T1548.002                    0  ...                    1                    0   
T1134                        0  ...                    0                    0   
T1134.002                    0  ...                    0                    0   
T1134.001                    0  ...                    0                    0   
...                        ...  ...                  ...                  ...   
T1102                        0  ...                    0                    0   
T1102.002                    0  ...                    0                    0   
T1102.001                    0  ...                    0                    0   
T1102.003                    0  ...                    0                    0   
T1047                        0  ...                    0                    0   

ID                       S0461                S0462                S0464  \
created    2020-06-01 00:00:00  2020-06-02 00:00:00  2020-06-02 00:00:00   
T1548.002                    0                    0                    0   
T1134                        0                    0         

In [5]:
# read in excel with newest 20 percent of techniques for evaluation
y_train = pd.read_excel("../Data for algorithms.xlsx", sheet_name="Y", index_col="ID")
y_train = y_train.drop(y_train.tail(2).index, axis=0)
y_train = y_train.drop(['created'], axis = 1).T
y_train

ID         S0471  S0470  S0469  S0473  S0475  S0477  S0476  S0481  S0482  \
T1548.002      0      0      0      0      0      0      0      0      0   
T1134          0      0      0      0      0      0      0      0      0   
T1134.002      0      0      0      0      0      0      0      0      0   
T1134.001      0      0      0      0      0      0      0      0      0   
T1087.002      0      0      0      0      0      0      1      0      0   
...          ...    ...    ...    ...    ...    ...    ...    ...    ...   
T1102          0      0      0      0      0      0      0      0      0   
T1102.002      0      0      0      0      0      0      0      0      0   
T1102.001      0      0      0      0      0      0      0      0      0   
T1102.003      0      0      0      0      0      0      0      0      0   
T1047          0      0      0      0      0      0      1      0      0   

ID         S0483  ...  S0590  S0593  S0591  S0592  S0595  S0594  S0596  S0597  \
T1548.002      0  ...      0      0      0      0      0      0      0      0   
T1134          0  ...      0      0      0      0      0      0      0      0   
T1134.002      0  ...      0      0      0      0      0      0      0      0   
T1134.001      0  ...      0      0      0      0      0      0      0      0   
T1087.002      1  ...      0      0      0      0      0      0      0      0   
...          ...  ...    ...    ...    ...    ...    ...    ...    ...    ...   
T1102          0  ...      0      0      0      0      0      0      0      0   
T1102.002      0  ...      0      0      0      0      0      0      0      0   
T1102.001      0  ...      0      0      0      0      0      0      0      0   
T1102.003      0  ...      0      0      0      0      0      0      0      0   
T1047          1  ...      0      0      0      0      0      0      0      0   

ID         S0599  S0600  
T1548.002      0      0  
T1134          0      0  
T1134.002      0      0  
T1134.001      0      0  
T1087.002      0      0  
...          ...    ...  
T1102          0      1  
T1102.002      0      0  
T1102.001      0      0  
T1102.003      0      0  
T1047          0      0  

[192 rows x 89 columns]

#### Reduce X to only single software per date

In [9]:
created = X.T["created"]
singlePoint = created.value_counts() [created.value_counts() == 1]
singlePoint.shape, singlePoint.index

((42,),
 DatetimeIndex(['2020-06-01', '2020-06-08', '2020-05-11', '2019-02-18',
                '2019-10-07', '2020-05-13', '2019-03-04', '2020-04-29',
                '2019-06-24', '2019-05-30', '2020-04-27', '2019-03-26',
                '2019-09-03', '2019-05-02', '2020-05-22', '2019-04-15',
                '2020-05-21', '2019-06-21', '2020-06-09', '2019-08-26',
                '2019-09-12', '2019-05-28', '2019-07-29', '2019-08-29',
                '2019-05-14', '2019-06-14', '2020-05-20', '2019-06-17',
                '2019-10-05', '2019-06-20', '2020-06-10', '2019-02-19',
                '2019-09-24', '2020-05-29', '2020-05-01', '2019-03-11',
                '2019-09-13', '2019-05-29', '2020-06-04', '2019-07-02',
                '2019-02-14', '2019-07-09'],
               dtype='datetime64[ns]', freq=None))

In [10]:
X_short = X.T.set_index(["created"])
#X.T.set_index(["created"]) == singlePoint.index
X_short = X_short.T[singlePoint.index]
X_short = X_short.T.sort_values(by=["created"]).T
X_short

created   2019-02-14 2019-02-18 2019-02-19 2019-03-04 2019-03-11 2019-03-26  \
T1548.002          0          0          0          0          1          0   
T1134              0          0          0          0          1          0   
T1134.002          0          0          0          0          1          0   
T1134.001          0          0          0          0          0          0   
T1087.002          0          0          0          0          1          0   
...              ...        ...        ...        ...        ...        ...   
T1102              0          0          0          0          0          0   
T1102.002          0          0          0          0          1          0   
T1102.001          0          0          0          0          0          0   
T1102.003          0          0          0          0          0          0   
T1047              0          0          0          0          1          1   

created   2019-04-15 2019-05-02 2019-05-14 2019-05-28  ... 2020-05-13  \
T1548.002          0          0          0          0  ...          0   
T1134              0          0          0          0  ...          1   
T1134.002          0          0          0          0  ...          0   
T1134.001          0          0          0          0  ...          0   
T1087.002          0          0          0          0  ...          0   
...              ...        ...        ...        ...  ...        ...   
T1102              0          0          0          0  ...          0   
T1102.002          0          1          0          0  ...          0   
T1102.001          0          0          0          0  ...          0   
T1102.003          0          0          0          0  ...          0   
T1047              0          0          1          1  ...          0   

created   2020-05-20 2020-05-21 2020-05-22 2020-05-29 2020-06-01 2020-06-04  \
T1548.002          0          0          0          0          0          0   
T1134              0          0          0          0          0          0   
T1134.002          0          0          0          0          0          0   
T1134.001          0          0          0          0          0          0   
T1087.002          0          0          0          0          0          0   
...              ...        ...        ...        ...        ...        ...   
T1102              0          0          0          0          0          0   
T1102.002          0          0          0          0          0          0   
T1102.001          0          0          0          0          0          0   
T1102.003          0          0          0          0          0          0   
T1047              0          0          0          0          0          0   

created   2020-06-08 2020-06-09 2020-06-10  
T1548.002          0          0          0  
T1134              0          0          0  
T1134.002          0          0          0  
T1134.001          0          0          0  
T1087.002          0          0          0  
...              ...        ...        ...  
T1102              0          0          0  
T1102.002          0          0          0  
T1102.001          0          0          0  
T1102.003          0          0          0  
T1047              0          0          0  

[192 rows x 42 columns]

#### VAR(n) analysis

In [13]:
from statsmodels.tsa.api import VAR

In [18]:
lags = 5 #change lags here
number_of_predictions = 30

In [19]:
# get newest n software of training set
lastX = X_short.T.tail(lags).T
lastX = np.asarray(lastX, dtype=int)
lastX.shape

(192, 5)

#### Fit Var(n) 

In [20]:
fitvar1= VAR(np.asarray(X_short.T, dtype=int)).fit(maxlags=lags)
fitvar1.k_ar # double check if right lag

5

#### Forecast m software

In [21]:
var_1_prediction = fitvar1.forecast(lastX.T, number_of_predictions)
var_1_prediction

array([[ 0.        ,  0.03735873,  0.0948971 , ...,  0.        ,
         0.        , -0.03154663],
       [ 0.        ,  0.09211233,  0.10055947, ...,  0.        ,
         0.        , -0.02255222],
       [ 0.        ,  0.00579239,  0.04930056, ...,  0.        ,
         0.        , -0.02378773],
       ...,
       [ 0.        , -0.01968299,  0.06765928, ...,  0.        ,
         0.        , -0.00883284],
       [ 0.        ,  0.02794371,  0.02588339, ...,  0.        ,
         0.        , -0.00687497],
       [ 0.        ,  0.02245845, -0.02308045, ...,  0.        ,
         0.        , -0.00582922]])

In [22]:
# replace float values by 0 or 1 with threshold 0.5
var_1_prediction = pd.DataFrame(np.where(var_1_prediction > 0.5, 1, 0)).T
var_1_prediction

0   1   2   3   4   5   6   7   8   9   ...  20  21  22  23  24  25  26  \
0     0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   0   
1     0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   0   
2     0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   0   
3     0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   0   
4     0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   0   
..   ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ...  ..  ..  ..  ..  ..  ..  ..   
187   0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   0   
188   0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   0   
189   0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   0   
190   0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   0   
191   0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   0   

     27  28  29  
0     0   0   0  
1     0   0   0  
2     0   0   0  
3     0   0   0  
4     0   0   0  
..   ..  ..  ..  
187   0   0   0  
188   0   0   0  
189   0   0   0  
190   0   0   0  
191   0   0   0  

[192 rows x 30 columns]

#### Evaluate result by F-score

In [23]:
def calculate_F_score(y_train: pd.DataFrame, predicted_sw: np.array, do_print=False):    

    assert y_train.shape[0] == predicted_sw.shape[0]
    
    ones = np.ones(len(y_train))

    occurences = np.array(predicted_sw).T @ ones
    
    F1_list = []
    Software_index = 0
    
    for software in y_train.columns:
        sum_s = y_train[software] @ predicted_sw

        precision = 0
        if occurences != 0:
            precision = sum_s/occurences
        
        number_of_techniques_y = y_train[software] @ ones
        recall = 0
        if number_of_techniques_y != 0:
            recall = sum_s/(number_of_techniques_y)
        F1 = 0
        if (precision + recall) != 0:
            F1 = 2 * precision* recall /(precision + recall)
            
        F1_list.append(F1)
    
    return max(F1_list), F1_list.index(max(F1_list)) ,predicted_sw

In [ ]:
# loop through all predictions and calculate F-score
i = 0
F1_values = pd.DataFrame([])
maxIndexValues = pd.DataFrame([])
for predict in var_1_prediction.columns:

    F1, maxIndex, generated_software = calculate_F_score(y_train, np.array(var_1_prediction[predict]))

    print(F1)
    F1_values[f"software_{i}"] = np.array([F1])
    maxIndexValues[f"software_{i}"] = np.array([maxIndex])
    i = i + 1

Saves predicted software together with the corresponding F-score and its related index of software in test set ino excel



In [ ]:
#var_1_prediction.append(F1_values).append(maxIndexValues).to_excel(f'VAR_{lags}_30_pred_final_result.xlsx') # uncommand to save

## Aggregate Software per year
Due to overlapping Date information, we calculate the relative frequency of techniques per year.

The resulting matrix is 192 x 4. The result looks like there are trends hidden. But from 4 data points and this high dimensionality, it is hard to find a pattern. This approach was not further investigated.

In [24]:
years = [2017, 2018, 2019, 2020]
yearDF = pd.DataFrame([])

for year in years:
    
    yearDF[f"{year}"] = X.T[created.dt.year == year].sum()
    total = yearDF[f"{year}"].sum()
    print(total)
    yearDF[f"{year}"] = yearDF[f"{year}"] / total
yearDF

1252.0
1282.0
1053.0
595.0


2017     2018      2019      2020
T1548.002  0.006390  0.00468  0.003799  0.003361
T1134      0.001597  0.00156  0.001899  0.001681
T1134.002  0.000000  0.00078  0.004748  0.001681
T1134.001  0.001597  0.00156  0.000000  0.003361
T1087.002  0.003195  0.00234  0.001899  0.000000
...             ...      ...       ...       ...
T1102      0.000000  0.00078  0.000950  0.000000
T1102.002  0.005591  0.00780  0.003799  0.000000
T1102.001  0.003994  0.00000  0.001899  0.000000
T1102.003  0.001597  0.00000  0.000000  0.001681
T1047      0.003994  0.01014  0.016144  0.003361

[192 rows x 4 columns]

In [ ]:
yearDF["sum"] = yearDF["sum"] * 0 # reset sum column for next cell run
yearDF["sum"] = yearDF.sum(axis=1)
yearDF.mean()

In [ ]:
yearDF_reduced = yearDF[yearDF["sum"] > yearDF["sum"].mean()]
yearDF_reduced.shape

In [ ]:
#yearDF.to_excel("yearly-props.xlsx") # uncommand to save

In [ ]:
#yearDF_reduced.to_excel("yearly-props-reduced.xlsx") # uncommand to save

In [ ]:
yearDF["2020"]

In [ ]:
import statsmodels.regression.linear_model as linMod
import statsmodels.api as sm

In [ ]:
y_data_test = pd.DataFrame(yearDF["2020"])
y_data = pd.DataFrame(yearDF["2019"])
X_data = sm.add_constant(yearDF.drop(["2020", "2019"], axis=1))

In [ ]:
ols_model = linMod.OLS(y_data, X_data)
ols_result = ols_model.fit()
ols_result.summary()

In [ ]:
X_test = sm.add_constant(yearDF.drop(["2020", "2017"], axis=1))
ols_pred = ols_result.predict(X_test)
ols_pred

In [ ]:
ols_analysis = pd.DataFrame(ols_pred, columns=["pred"])
ols_analysis["test"] = y_data_test
ols_analysis["diff"] = ols_analysis["pred"] - ols_analysis["test"]
ols_analysis["diff_relative"] = ols_analysis["diff"] / ols_analysis["test"]
ols_analysis

In [ ]:
# OLS for each technique

In [ ]:
first_row = yearDF_reduced.drop("sum", axis=1).head(1).T
first_row["lag"] = first_row["T1087.001"].shift(1)
first_row = first_row[1:]
first_row

In [ ]:
X_ols = sm.add_constant(first_row["lag"])
y_ols = pd.DataFrame(first_row["T1087.001"])
#X_train_ols = 
#y_tain_ols = 

In [ ]:
myModel = sm.OLS(y_ols, X_ols)
myResult = myModel.fit()
myResult.summary()